In [ ]:
!pip install -r requirements.txt

In [ ]:
# БЛОК 1
import os
import requests
import re
import pandas as pd
import pdfplumber
import numpy as np
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import time
import pymorphy3
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter
import nltk
import warnings
import logging
import ast

# Отображение графиков в ноутбуке
%matplotlib inline

# Отключение предупреждений
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", message=".*datetime.utcnow.*")
os.environ["PYTHONWARNINGS"] = "ignore::DeprecationWarning"
warnings.filterwarnings("ignore", message=".*multi-threaded, use of fork().*")

# Отключаем логирование для модуля pdfminer
logging.getLogger("pdfminer").setLevel(logging.ERROR)

# Загрузка ресурсов NLTK
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

# Настройка локальных путей
# Получаем текущую директорию
current_dir = os.getcwd()

# Создаем папку project_data рядом
BASE_DIR = os.path.join(current_dir, "project_data")
PDF_FOLDER = os.path.join(BASE_DIR, "pdfs")
OUTPUT_CSV = os.path.join(BASE_DIR, "dataset_final.csv")
TOPIC_CSV = os.path.join(BASE_DIR, "dataset_with_topics.csv")

# Создаем папки физически
os.makedirs(PDF_FOLDER, exist_ok=True)

print(f"Рабочая папка проекта: {BASE_DIR}")
print(f"Папка для PDF: {PDF_FOLDER}")

In [ ]:
# БЛОК 2 Базовые настройки
base_url = "https://imt-journal.ru"
archive_url = f"{base_url}/archive"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

def download_pdf(pdf_url, folder):
    filename = os.path.join(folder, pdf_url.split("/")[-1])
    if os.path.exists(filename):
        print(f"Файл уже есть: {filename}")
        return False

    try:
        print(f"Скачивание: {pdf_url.split('/')[-1]}")
        response = requests.get(pdf_url, headers=headers, timeout=30)
        response.raise_for_status()
        with open(filename, "wb") as f:
            f.write(response.content)
        return True
    except Exception as e:
        print(f"Ошибка скачивания {pdf_url}: {e}")
        return False

def get_pdfs_from_page(page_url):
    pdf_urls = []
    try:
        response = requests.get(page_url, headers=headers, timeout=30)
        soup = BeautifulSoup(response.text, "html.parser")
        for a in soup.find_all("a", href=True):
            if a['href'].endswith(".pdf"):
                full_url = urljoin(base_url, a['href'])
                if full_url not in pdf_urls:
                    pdf_urls.append(full_url)
    except Exception as e:
        print(f"Ошибка страницы {page_url}: {e}")
    return pdf_urls

def run_scraper():
    print("ЗАПУСК СБОРА PDF")
    visited_pages = set()
    current_page = archive_url
    page_num = 1
    max_pages = 50

    # Скачиваем PDF с текущей страницы
    while current_page and current_page not in visited_pages and page_num <= max_pages:
        print(f"\nСтраница {page_num}: {current_page}")
        visited_pages.add(current_page)

        pdfs = get_pdfs_from_page(current_page)
        print(f"Найдено PDF: {len(pdfs)}")

        for pdf_url in pdfs:
            download_pdf(pdf_url, PDF_FOLDER)
            time.sleep(0.5) #Пауза, чтобы не дудосить сайт

        #Ищем следующую страницу
        try:
            response = requests.get(current_page, headers=headers, timeout=30)
            soup = BeautifulSoup(response.text, "html.parser")
            next_link = None
            pagination = soup.find(class_='pagination')
            if pagination:
                curr = pagination.find('span', class_='current')
                if curr and curr.find_next_sibling('a'):
                    next_link = curr.find_next_sibling('a')['href']

            if not next_link:
                 for a in soup.find_all('a', href=True):
                    if a.get_text(strip=True) in ['>', '»', 'Вперед', 'Next', 'Следующая']:
                        next_link = a['href']
                        break

            if next_link:
                next_page_url = urljoin(current_page, next_link)
                if next_page_url == current_page or next_page_url in visited_pages:
                    break
                current_page = next_page_url
                page_num += 1
            else:
                print("Последняя страница.")
                break

        except Exception as e:
            print(f"Ошибка пагинации: {e}")
            break

# Запускаем скрейпер
run_scraper()

In [ ]:
# БЛОК 3 Парсинг, обработка текста и создание датаета
import tqdm

print("НАЧАЛО ОБРАБОТКИ PDF И СОЗДАНИЯ ДАТАСЕТА")

# Инициализация морфологического анализатора и стоп-слов
morph = pymorphy3.MorphAnalyzer()
stop_words = set(stopwords.words('russian'))

# Расширенные стоп-слова
academic_stopwords = [ 'это', 'наш', 'год', 'рис', 'табл', 'стр', 'см', 'статья', 'работа', 'исследование', 'данные', 'результат', 'метод', 'основа', 'использование', 'показать', 'рассмотреть', 'являться', 'предложить', 'провести', 'задача', 'рисунок', 'таблица', 'схема', 'цель', 'проблема', 'решение', 'пример', 'случай', 'вид', 'тип', 'сравнение', 'оценка', 'вопрос', 'подход', 'номер', 'онтология', 'применение', 'область', 'научный', 'новый', 'современный', 'различный', 'получить', 'выполнить', 'представить', 'описать', 'существовать', 'позволять', 'введение', 'заключение', 'вывод', 'список', 'литература', 'библиография', 'аннотация', 'ключевые', 'слова', 'удк', 'doi', 'том', 'выпуск', 'автор', 'данный', 'использовать', 'свойство', 'разработка', 'определение', 'параметр', 'ключевой', 'который', 'свой', 'этот', 'также', 'каждый', 'мочь', 'весь']
stop_words.update(academic_stopwords)

def tokenize_for_lda(text):
    if not text: return []

    # Склеивание переносов слов
    text = re.sub(r'(\w+)-\s*\n\s*(\w+)', r'\1\2', text)

    # Удаление всего кроме букв
    text = re.sub(r'[^а-яёА-ЯЁ\s]', ' ', text)

    # Приведение к нижнему регистру и удаление лишних пробелов
    text = re.sub(r'\s+', ' ', text).strip().lower()

    tokens = []
    for word in text.split():
        if len(word) <= 2: continue

        # Морфологический анализ
        p = morph.parse(word)[0]

        # Фильтрация по частям речи для LDA (существительные и прилагательные)
        if p.tag.POS not in {'NOUN', 'ADJF'}: continue

        lemma = p.normal_form
        if lemma not in stop_words:
            tokens.append(lemma)
    return tokens

# Извлекает текст из PDF с обрезкой колонтитулов
def extract_text_with_crop(pdf_path):
    full_text = ""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                width, height = page.width, page.height
                cropped_page = page.crop((0, 60, width, height - 60))
                page_text = cropped_page.extract_text()
                if page_text:
                    full_text += page_text + "\n"
    except Exception as e:
        print(f"Ошибка чтения {pdf_path}: {e}")
        return ""
    return full_text

# Разбивает текст на статьи по УДК и выделяет смысловую часть
def parse_articles_from_text(raw_text, filename):
    articles_data = []

    # Разбиение полного текста файла на куски по маркеру УДК
    chunks = re.split(r'\n(?=УДК\s+[\d\.\+\:]+)', raw_text)

    for chunk in chunks:
        # Пропускаем слишком короткие фрагменты
        if len(chunk) < 1000: continue

        udc_match = re.search(r'УДК\s+([\d\.\+\:]+)', chunk)
        udc = udc_match.group(1) if udc_match else "N/A"

        # Извлечение аннотации
        annot_match = re.search(r'Аннотация\.?(.*?)(Ключевые слова|Keywords)', chunk, re.DOTALL | re.IGNORECASE)
        annotation = annot_match.group(1).strip() if annot_match else ""

        # Извлечение ключевых слов
        kw_match = re.search(r'Ключевые слова[:\.](.*?)(Цитирование|Citation|Введение|Introduction)', chunk, re.DOTALL | re.IGNORECASE)
        keywords = kw_match.group(1).strip() if kw_match else ""

        # Поиск начала основного текста
        intro_match = re.search(r'(?:\n|^)\s*(?:1\.?)?\s*(?:Введение|Introduction)(?:\.|:)?\s+', chunk, re.IGNORECASE)
        main_text_raw = ""

        if intro_match:
            start_pos = intro_match.end()
            main_text_raw = chunk[start_pos:]
        else:
            fallback_match = re.search(r'(Ключевые слова|Keywords|Цитирование|Citation).*?(\n\s*\n|\n[А-Я])', chunk, re.DOTALL | re.IGNORECASE)
            if fallback_match:
                start_pos = fallback_match.end()
                main_text_raw = chunk[start_pos:]
            else:
                continue

        # Поиск конца текста (обрезаем текст до списка литературы)
        ref_start = re.search(r'(?:\n|^)\s*(?:СПИСОК ЛИТЕРАТУРЫ|Список источников|REFERENCES|Библиографический список)', main_text_raw, re.IGNORECASE)
        if ref_start:
            main_text_raw = main_text_raw[:ref_start.start()]

        # Токенизация и проверка на длину
        lda_tokens = tokenize_for_lda(main_text_raw)

        if len(lda_tokens) > 50:
            articles_data.append({ 'source': filename, 'udc': udc, 'annotation': annotation, 'keywords': keywords, 'token_len': len(lda_tokens),
                                   'main_text_sample': main_text_raw[:200].replace('\n', ' ') + "...", 'lda_tokens': lda_tokens})
    return articles_data

# Запуск
all_articles = []
if os.path.exists(OUTPUT_CSV):
    print("Найден существующий CSV, читаем его")
    df = pd.read_csv(OUTPUT_CSV)
    # Конвертация строки списка обратно в список
    df['lda_tokens'] = df['lda_tokens'].apply(ast.literal_eval)
else:
    pdf_files = [f for f in os.listdir(PDF_FOLDER) if f.endswith(".pdf")]
    if not pdf_files:
        print("PDF файлы не найдены в папке:", PDF_FOLDER)
        df = pd.DataFrame() # Пустой датафрейм чтобы не упало дальше
    else:
        for pdf_file in tqdm.tqdm(pdf_files, desc="Парсинг PDF"):
            full_path = os.path.join(PDF_FOLDER, pdf_file)
            raw_pdf_text = extract_text_with_crop(full_path)
            if raw_pdf_text:
                articles = parse_articles_from_text(raw_pdf_text, pdf_file)
                all_articles.extend(articles)

        # Создание DataFrame
        df = pd.DataFrame(all_articles)
        if not df.empty:
            cols = ['source', 'udc', 'token_len', 'annotation', 'keywords', 'main_text_sample', 'lda_tokens']
            df = df[cols]
            df.to_csv(OUTPUT_CSV, index=False)
            print(f"Готово! Статей в базе: {len(df)}")
            print(f"Пример данных (первые 5 строк):")
            print(df[['main_text_sample']].head())
        else:
            print("Валидные статьи не сформированы.")

In [ ]:
# БЛОК 4
if df is not None and not df.empty:
    print(f"Всего статей: {len(df)}")

    # Сбор всех слов
    all_words = []
    for tokens in df['lda_tokens']:
        if isinstance(tokens, str):
            tokens = ast.literal_eval(tokens)
        all_words.extend(tokens)

    # Гистограмма Топ-20
    word_freq = Counter(all_words)
    common_words = word_freq.most_common(20)

    plt.figure(figsize=(12, 6))
    plt.bar([x[0] for x in common_words], [x[1] for x in common_words], color='steelblue')
    plt.title("Топ-20 слов во всем корпусе")
    plt.xticks(rotation=45)
    plt.show()

    # Глобальное облако слов
    print("Генерация глобального облака слов")
    text_cloud = " ".join(all_words)
    wordcloud = WordCloud(width=1600, height=800, background_color='white', colormap='viridis').generate(text_cloud)

    plt.figure(figsize=(14, 7))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title("Облако слов (Весь датасет)", fontsize=16)
    plt.show()

else:
    print("Нет данных.")

In [ ]:
# БЛОК 5 Тематическое моделирование LDA
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel, LdaMulticore
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# Отключаем логи Gensim
logging.getLogger("gensim").setLevel(logging.ERROR)

if df is not None and not df.empty:
    print(f"Всего статей для анализа: {len(df)}")

    # 1. Подготовка данных
    data_words = list(df['lda_tokens'])

    # Проверка формата (список списков)
    if isinstance(data_words[0], str):
         data_words = [ast.literal_eval(x) for x in data_words]

    # Создание биграмм
    bigram = gensim.models.Phrases(data_words, min_count=5, threshold=10)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    data_ready = [bigram_mod[doc] for doc in data_words]

    # Создание словаря
    id2word = corpora.Dictionary(data_ready)
    id2word.filter_extremes(no_below=3, no_above=0.5)
    corpus = [id2word.doc2bow(text) for text in data_ready]

    # Поиск оптимального количества тем
    def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=1):
        coherence_values = []
        model_list = []
        for num_topics in range(start, limit, step):
            print(f"Обучение модели на {num_topics} тем")
            model = LdaMulticore(corpus=corpus, id2word=dictionary, num_topics=num_topics, random_state=42, passes=10, workers=1, per_word_topics=False)
            model_list.append(model)
            coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
            coherence_values.append(coherencemodel.get_coherence())
        return model_list, coherence_values

    print("Поиск тем")
    start_t, limit_t, step_t = 3, 9, 1
    model_list, coherence_values = compute_coherence_values(id2word, corpus, data_ready, limit=limit_t, start=start_t, step=step_t)

    # Визуализация графика Coherence
    plt.figure(figsize=(10, 5))
    plt.plot(range(start_t, limit_t, step_t), coherence_values, marker='o')
    plt.xlabel("Количество тем")
    plt.ylabel("Coherence score")
    plt.show()

    # Выбор лучшей модели
    best_index = np.argmax(coherence_values)
    lda_model = model_list[best_index]
    num_topics = range(start_t, limit_t, step_t)[best_index]
    best_coherence = coherence_values[best_index]
    print(f"Выбрана модель с {num_topics} темами (coherence score: {best_coherence:.4f}).")

     # Текстовый вывод тем
    print("КЛЮЧЕВЫЕ СЛОВА ПО ТЕМАМ")
    topics = lda_model.print_topics(num_words=10)
    for topic in topics:
        print(f"Тема {topic[0]}: {topic[1]}")

    # Визуализация облака слов
    print("Генерация облаков слов")
    cols = 2
    rows = (num_topics // cols) + (num_topics % cols > 0)
    fig, axes = plt.subplots(rows, cols, figsize=(16, 5 * rows))
    axes = axes.flatten()

    for i in range(num_topics):
        topic_words = dict(lda_model.show_topic(i, 30))
        cloud = WordCloud(background_color='white', width=600, height=400, colormap='tab10').generate_from_frequencies(topic_words)
        axes[i].imshow(cloud)
        axes[i].set_title(f'Тема {i}', fontsize=16)
        axes[i].axis('off')

    # Скрываем пустые графики
    for j in range(num_topics, len(axes)):
        axes[j].axis('off')

    plt.tight_layout()
    plt.show()

    # Сохранение результатов
    def get_main_topic_df(model, corpus):
        results = []
        topics_per_doc = model.get_document_topics(corpus)
        for doc_topics in topics_per_doc:
            sorted_topics = sorted(doc_topics, key=lambda x: x[1], reverse=True)
            if sorted_topics:
                results.append(sorted_topics[0])
            else:
                results.append((None, 0.0))
        return results

    topic_info = get_main_topic_df(lda_model, corpus)
    df['dominant_topic'] = [int(x[0]) if x[0] is not None else -1 for x in topic_info]
    df['topic_confidence'] = [float(x[1]) for x in topic_info]

    # Карта тем
    topic_mapping = {i: ", ".join([w[0] for w in lda_model.show_topic(i, 5)]) for i in range(num_topics)}
    df['topic_keywords'] = df['dominant_topic'].map(topic_mapping)

    df.to_csv(TOPIC_CSV, index=False)
    print(f"Файл сохранен: {TOPIC_CSV}")

    # Интерактивная визуализация
    pyLDAvis.enable_notebook()
    vis = gensimvis.prepare(lda_model, corpus, id2word)
    display(vis)

else:
    print("Датафрейм пуст.")